# 01 — EDA and Preprocessing
Goal: inspect raw interactions/metadata, pick column names, filter sparse users/items, and create train/test splits for downstream models.


In [1]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)


In [2]:
import os

os.chdir('/home/alyx/Documents/RS/Project')

In [3]:
import pandas as pd
from pathlib import Path

from src import config
from src.data_loading import load_interactions, load_games_metadata, merge_datasets
from src.preprocessing import filter_users_and_items, train_test_split_by_time

RAW_INTERACTIONS = config.INTERACTIONS_FILE
RAW_GAMES = config.GAMES_METADATA_FILE
USER_COL = config.USER_COL
ITEM_COL = config.ITEM_COL
TIMESTAMP_COL = config.TIMESTAMP_COL
INTERACTION_VALUE_COL = config.INTERACTION_VALUE_COL
GAME_ID_COL_IN_GAMES = config.GAME_ID_COL_IN_GAMES


## Load raw datasets

### Quick EDA
Basic stats for users/items/timestamps to guide column choices and thresholds.


In [5]:
interactions_raw = load_interactions(RAW_INTERACTIONS)
games_raw = load_games_metadata(RAW_GAMES)


Loaded interactions: (11258715, 2)
Columns: ['playerid', 'appid']
            playerid  appid
0  76561198060698936     60
1  76561198060698936   1670
2  76561198060698936   3830
3  76561198060698936   1600
4  76561198060698936   2900
Loaded games metadata: (89618, 47)
Columns: ['appid', 'name', 'release_date', 'required_age', 'price', 'dlc_count', 'detailed_description', 'about_the_game', 'short_description', 'reviews', 'header_image', 'website', 'support_url', 'support_email', 'windows', 'mac', 'linux', 'metacritic_score', 'metacritic_url', 'achievements', 'recommendations', 'notes', 'supported_languages', 'full_audio_languages', 'packages', 'developers', 'publishers', 'categories', 'genres', 'screenshots', 'movies', 'user_score', 'score_rank', 'positive', 'negative', 'estimated_owners', 'average_playtime_forever', 'average_playtime_2weeks', 'median_playtime_forever', 'median_playtime_2weeks', 'discount', 'peak_ccu', 'tags', 'pct_pos_total', 'num_reviews_total', 'pct_pos_recent', 'num

In [6]:
print("Interactions columns:", interactions_raw.columns.tolist())
print("Games columns:", games_raw.columns.tolist())
print("User counts (raw):", interactions_raw[USER_COL].nunique())
print("Item counts (raw):", interactions_raw[ITEM_COL].nunique())
print("Top users:")
print(interactions_raw[USER_COL].value_counts().head())
print("Top items:")
print(interactions_raw[ITEM_COL].value_counts().head())
if TIMESTAMP_COL and TIMESTAMP_COL in interactions_raw.columns:
    ts = pd.to_datetime(interactions_raw[TIMESTAMP_COL], errors='coerce')
    print("Timestamp coverage:", ts.min(), ts.max())
    print("Missing timestamps:", ts.isna().sum())


Interactions columns: ['playerid', 'appid']
Games columns: ['appid', 'name', 'release_date', 'required_age', 'price', 'dlc_count', 'detailed_description', 'about_the_game', 'short_description', 'reviews', 'header_image', 'website', 'support_url', 'support_email', 'windows', 'mac', 'linux', 'metacritic_score', 'metacritic_url', 'achievements', 'recommendations', 'notes', 'supported_languages', 'full_audio_languages', 'packages', 'developers', 'publishers', 'categories', 'genres', 'screenshots', 'movies', 'user_score', 'score_rank', 'positive', 'negative', 'estimated_owners', 'average_playtime_forever', 'average_playtime_2weeks', 'median_playtime_forever', 'median_playtime_2weeks', 'discount', 'peak_ccu', 'tags', 'pct_pos_total', 'num_reviews_total', 'pct_pos_recent', 'num_reviews_recent']
User counts (raw): 46941
Item counts (raw): 40988
Top users:
playerid
76561198094227663    32463
76561197962473290    28196
76561197976968076    25707
76561198027214426    25100
76561198842864763    25

## Inspect and choose columns
Update `src/config.py` after deciding the correct column names for:
- `USER_COL`, `ITEM_COL`
- `TIMESTAMP_COL` (set to `None` if unavailable)
- `INTERACTION_VALUE_COL` (e.g., hours played / rating / implicit 1)


In [8]:
print("Interaction column value counts", interactions_raw[ITEM_COL].value_counts().head())
if TIMESTAMP_COL and TIMESTAMP_COL in interactions_raw.columns:
    print(interactions_raw[TIMESTAMP_COL].describe())


Interaction column value counts appid
730       43967
578080    28356
550       24727
218620    23785
304930    21620
Name: count, dtype: int64


## Merge and align IDs
Ensure item identifiers match between interactions and games metadata. Rename games metadata id column to match ITEM_COL.


In [9]:
interactions_clean, games_clean = merge_datasets(
    interactions_raw,
    games_raw,
    user_col=USER_COL,
    item_col=ITEM_COL,
    game_id_col_in_games=GAME_ID_COL_IN_GAMES,
)


After merge alignment: 9176497 interactions, 89618 unique games


## Filter rare users/items
Set thresholds in `config.MIN_USER_INTERACTIONS` / `config.MIN_ITEM_INTERACTIONS`.


### After filtering
Check remaining users/items and sparsity after applying min interaction thresholds.


In [11]:
interactions_filtered = filter_users_and_items(
    interactions_clean,
    min_user_interactions=config.MIN_USER_INTERACTIONS,
    min_item_interactions=config.MIN_ITEM_INTERACTIONS,
    user_col=USER_COL,
    item_col=ITEM_COL,
)
print("After filtering:", interactions_filtered.shape)


After filtering: (9161667, 2)


In [12]:
print("Filtered users:", interactions_filtered[USER_COL].nunique())
print("Filtered items:", interactions_filtered[ITEM_COL].nunique())
print("Interactions shape:", interactions_filtered.shape)

Filtered users: 44021
Filtered items: 31862
Interactions shape: (9161667, 2)


### Train/Test Split
Hold out the last (or random) interaction per user, inspect resulting shapes.


In [13]:
train_df, test_df = train_test_split_by_time(
    interactions_filtered,
    user_col=USER_COL,
    item_col=ITEM_COL,
    timestamp_col=TIMESTAMP_COL,
    n_test_items=1,
    random_state=config.RANDOM_STATE,
)
print("Train size", train_df.shape, "Test size", test_df.shape)


Train size (9117646, 2) Test size (44021, 2)


## Save processed data
Saved files feed later notebooks.


In [15]:
processed_dir = config.PROCESSED_DATA_DIR
processed_dir.mkdir(parents=True, exist_ok=True)

train_path = processed_dir / "train_interactions.parquet"
test_path = processed_dir / "test_interactions.parquet"
games_path = processed_dir / "games_metadata.parquet"

train_df.to_parquet(train_path, index=False)
test_df.to_parquet(test_path, index=False)
games_clean.to_parquet(games_path, index=False)

print("Saved:", train_path, test_path, games_path, sep="\n")


Saved:
/home/alyx/Documents/RS/Project/data/processed/train_interactions.parquet
/home/alyx/Documents/RS/Project/data/processed/test_interactions.parquet
/home/alyx/Documents/RS/Project/data/processed/games_metadata.parquet
